# LlamaIndex Hybrid Retriever + Reranking

Hybrid Retrievers are a great way to combine the strenghts of different retrievers. Combined with filtering and reranking, this can be especially powerful in retrieving only the most relevant context from multiple methods. TruLens can take us even farther to highlight the strengths of each component retriever along with measuring the success of the hybrid retriever

## Setup

In [2]:
# ! pip install trulens_eval==0.24.0 llama_index==0.10.11 llama-index-readers-file llama-index-llms-openai llama-index-retrievers-bm25 openai pypdf torch sentence-transformers

In [1]:
!pip install trulens_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.0/751.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.3/706.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.9 MB/

In [1]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk"

In [9]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Huggingface, Tru
#from trulens_eval.schema import FeedbackResult
from trulens_eval.schema.feedback import FeedbackResult
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


## Get data

In [10]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  34.6M      0 --:--:-- --:--:-- --:--:-- 34.5M


## Create index

In [18]:
!ls /path/to/llama_index/indices
!ls /path/to/llama_index/core

ls: cannot access '/path/to/llama_index/indices': No such file or directory
ls: cannot access '/path/to/llama_index/core': No such file or directory


In [19]:
!ls /usr/local/lib/python3.10/dist-packages/llama_index/indices
!ls /usr/local/lib/python3.10/dist-packages/llama_index/core


managed
agent		evaluation	    llama_pack	      py.typed			settings.py
async_utils.py	exec_utils.py	    llms	      query_engine		_static
base		extractors	    memory	      query_pipeline		storage
bridge		graph_stores	    multi_modal_llms  question_gen		text_splitter
callbacks	image_retriever.py  node_parser       readers			tools
chat_engine	img_utils.py	    objects	      response			types.py
command_line	indices		    output_parsers    response_synthesizers	utilities
composability	ingestion	    playground	      retrievers		utils.py
constants.py	__init__.py	    postprocessor     schema.py			vector_stores
data_structs	instrumentation     program	      selectors
download	langchain_helpers   prompts	      service_context_elements
embeddings	llama_dataset	    __pycache__       service_context.py


In [28]:
from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.legacy.retrievers.bm25_retriever import BM25Retriever  # Adjusted import path
from llama_index.core.indices.vector_store.retrievers.retriever import VectorIndexRetriever  # Adjusted import path
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI

splitter = SentenceSplitter(chunk_size=1024)

# load documents
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

nodes = splitter.get_nodes_from_documents(documents)

# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
)


## Set up retrievers

In [32]:
# retireve the top 10 most similar nodes using embeddings
vector_retriever = VectorIndexRetriever(index)

# retireve the top 10 most similar nodes using bm25
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)

## Create Hybrid (Custom) Retriever

In [33]:
from llama_index.core.retrievers import BaseRetriever

class HybridRetriever(BaseRetriever):
    def __init__(self, vector_retriever, bm25_retriever):
        self.vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        super().__init__()

    def _retrieve(self, query, **kwargs):
        bm25_nodes = self.bm25_retriever.retrieve(query, **kwargs)
        vector_nodes = self.vector_retriever.retrieve(query, **kwargs)

        # combine the two lists of nodes
        all_nodes = []
        node_ids = set()
        for n in bm25_nodes + vector_nodes:
            if n.node.node_id not in node_ids:
                all_nodes.append(n)
                node_ids.add(n.node.node_id)
        return all_nodes

index.as_retriever(similarity_top_k=5)

hybrid_retriever = HybridRetriever(vector_retriever, bm25_retriever)

## Set up reranker

In [36]:
from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(top_n=4, model="BAAI/bge-reranker-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [42]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever=hybrid_retriever,
    node_postprocessors=[reranker]
)

In [43]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 104.198.197.242



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Initialize Context Relevance checks

Include relevance checks for bm25, vector retrievers, hybrid retriever and the filtered hybrid retriever (after rerank and filter).

This requires knowing the feedback selector for each.

In [ ]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval.schema import Select
import numpy as np

# Initialize provider class
openai = OpenAI()

bm25_context = Select.RecordCalls._retriever.bm25_retriever.retrieve.rets[:].node.text
vector_context = Select.RecordCalls._retriever.vector_retriever._retrieve.rets[:].node.text
hybrid_context = Select.RecordCalls._retriever.retrieve.rets[:].node.text
hybrid_context_filtered = Select.RecordCalls._node_postprocessors[0]._postprocess_nodes.rets[:].node.text

# Question/statement relevance between question and each context chunk.
f_context_relevance_bm25 = (
    Feedback(openai.context_relevance, name = "BM25")
    .on_input()
    .on(bm25_context)
    .aggregate(np.mean)
    )

f_context_relevance_vector = (
    Feedback(openai.context_relevance, name = "Vector")
    .on_input()
    .on(vector_context)
    .aggregate(np.mean)
    )

f_context_relevance_hybrid = (
    Feedback(openai.context_relevance, name = "Hybrid")
    .on_input()
    .on(hybrid_context)
    .aggregate(np.mean)
    )

f_context_relevance_hybrid_filtered = (
    Feedback(openai.context_relevance, name = "Hybrid Filtered")
    .on_input()
    .on(hybrid_context_filtered)
    .aggregate(np.mean)
    )

## Add feedbacks

In [ ]:
tru_recorder = TruLlama(query_engine,
    app_id='Hybrid Retriever Query Engine',
    feedbacks=[f_context_relevance_bm25, f_context_relevance_vector, f_context_relevance_hybrid, f_context_relevance_hybrid_filtered])

In [ ]:
with tru_recorder as recording:
    response = query_engine.query("What is the impact of climate change on the ocean?")

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed